In [3]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [5]:
IMG_SIZE = 256
input_dir = "Plant_leave_diseases_dataset_without_augmentation"        # Original dataset path
output_dir = "augmented_dataset/"   # Augmented dataset path
target_count = 3000  # Desired number of images per class
os.makedirs(output_dir, exist_ok=True)

In [6]:
def preprocess(image):
    """Resize and normalize image to [0,1]."""
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    # image = image.astype("float32") / 255.0
    return image

In [7]:
def random_rotation(image, angle_range=45):
    h, w = image.shape[:2]
    angle = np.random.uniform(-angle_range, angle_range)
    M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1)
    return cv2.warpAffine(image, M, (w, h))

In [8]:
def random_translation(image, max_shift=0.2):
    h, w = image.shape[:2]
    tx = np.random.uniform(-max_shift, max_shift) * w
    ty = np.random.uniform(-max_shift, max_shift) * h
    M = np.float32([[1, 0, tx], [0, 1, ty]])
    return cv2.warpAffine(image, M, (w, h))

In [9]:
def random_shear(image, shear_range=15):
    h, w = image.shape[:2]
    pts1 = np.float32([[5,5],[w-5,5],[5,h-5]])
    pt1 = 5 + np.random.randint(-shear_range, shear_range)
    pt2 = 5 + np.random.randint(-shear_range, shear_range)
    pts2 = np.float32([[pt1,pt2],[w-pt1,5],[5,h-pt2]])
    M = cv2.getAffineTransform(pts1,pts2)
    return cv2.warpAffine(image,M,(w,h))

In [10]:
def random_crop(image, crop_scale=0.8):
    h, w = image.shape[:2]
    new_h, new_w = int(h * crop_scale), int(w * crop_scale)
    top = np.random.randint(0, h - new_h)
    left = np.random.randint(0, w - new_w)
    cropped = image[top:top+new_h, left:left+new_w]
    return cv2.resize(cropped, (w, h))

In [11]:
def augment_image(image):
    ops = [
        lambda x: random_rotation(x),
        lambda x: cv2.flip(x, 1),   # Horizontal Flip
        lambda x: cv2.flip(x, 0),   # Vertical Flip
        lambda x: random_translation(x),
        lambda x: random_shear(x),
        lambda x: random_crop(x)
    ]
    op = np.random.choice(ops)  # pick random transformation
    return op(image)

In [12]:
for class_name in os.listdir(input_dir):
    class_path = os.path.join(input_dir, class_name)
    save_class_path = os.path.join(output_dir, class_name)
    os.makedirs(save_class_path, exist_ok=True)

    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    count_existing = len(images)

    print(f"\nProcessing {class_name} (current: {count_existing}, target: {target_count})")

    # Copy original images first
    for img_name in images:
        img_path = os.path.join(class_path, img_name)
        image = cv2.imread(img_path)
        if image is not None:
            cv2.imwrite(os.path.join(save_class_path, img_name), image)
    
    # Augment until reaching target_count
    idx = 0
    pbar = tqdm(total=target_count - count_existing, desc=f"Augmenting {class_name}")
    while count_existing < target_count:
        img_name = images[idx % len(images)]  # cycle through originals
        img_path = os.path.join(class_path, img_name)
        image = cv2.imread(img_path)
        if image is None:
            idx += 1
            continue

        aug_img = augment_image(image)
        save_name = f"{os.path.splitext(img_name)[0]}_aug_{count_existing}.jpg"
        save_path = os.path.join(save_class_path, save_name)
        cv2.imwrite(save_path, aug_img)

        count_existing += 1
        idx += 1
        pbar.update(1)
    pbar.close()    


Processing Tomato___Bacterial_spot (current: 2127, target: 3000)


Augmenting Tomato___Bacterial_spot: 100%|██████████| 873/873 [00:01<00:00, 595.54it/s]



Processing Tomato___Early_blight (current: 1000, target: 3000)


Augmenting Tomato___Early_blight: 100%|██████████| 2000/2000 [00:02<00:00, 761.30it/s]



Processing Tomato___healthy (current: 1591, target: 3000)


...

Augmenting Tomato___Target_Spot: 100%|██████████| 1596/1596 [00:02<00:00, 728.43it/s]



Processing Tomato___Tomato_mosaic_virus (current: 373, target: 3000)


Augmenting Tomato___Tomato_mosaic_virus: 100%|██████████| 2627/2627 [00:03<00:00, 739.46it/s]



Processing Tomato___Tomato_Yellow_Leaf_Curl_Virus (current: 5357, target: 3000)


Augmenting Tomato___Tomato_Yellow_Leaf_Curl_Virus: 0it [00:00, ?it/s]


🌱 Geometric Transformations in Plant Disease Detection

In our project, we apply a series of geometric image transformations to expand the dataset and improve the robustness of our CNN model.

1. Rotation

The leaf can appear at any angle when photographed.

Random rotation ensures the model learns to recognize disease patterns regardless of orientation.

2. Horizontal Flip

Simulates the scenario where a leaf is turned left or right.

Helps the model become invariant to left–right orientation.

3. Vertical Flip

Covers cases where a leaf might be captured upside down.

Increases diversity in orientation.

4. Translation (Shifting)

Mimics leaves that are off-center in the image.

Makes the model robust to disease spots appearing at different positions.

5. Scaling / Resizing

Represents zoom-in and zoom-out variations of leaf images.

Ensures the model learns from both close-up disease patches and full-leaf views.

6. Shearing (Affine Distortion)

Simulates perspective distortion when leaves are not flat or are photographed at an angle.

Improves the model’s ability to generalize to real-world photos.

🌾 Why These Transformations Help

Increase the size of the dataset without collecting new images.

Prevent the CNN from overfitting to fixed orientations or positions.

Make the model robust to real-world conditions where farmers may capture leaves at different angles, zoom levels, and perspectives.

Ultimately, they improve generalization and accuracy in plant disease classification.